In [24]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
from src.utilis import load_object
import shap
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
# Data Transformation : Transformation des data
from src.components.data_transformation import DataTransformation

In [5]:
# Data


transformer_row_data= DataTransformation()
X_train,X_test, y_train, y_test = transformer_row_data.get_data_frames()

In [6]:
MODEL_PATH = "C:/Users/Lenovo/Documents/DSPython/projetscoring/notebook/artifacts/best_model_pretrained.pkl"
PREPROCESSOR_PATH = "C:/Users/Lenovo/Documents/DSPython/projetscoring/notebook/artifacts/preprocessor.pkl"
model = load_object(MODEL_PATH)
preprocessor = load_object(PREPROCESSOR_PATH)

In [11]:
model

LogisticRegression(C=0.001, solver='saga')

In [33]:
# preprocessor.get_feature_names_out ()

In [76]:
X_train_ = X_train.replace((np.inf, -np.inf), np.nan).reset_index(drop=True)
X_train_scaled = preprocessor.transform(X_train_)
feature_names_X_train = preprocessor.get_feature_names_out ()
X_train = pd.DataFrame(X_train_scaled, columns=feature_names_X_train)
X_train

,num_pipeline__CODE_GENDER,num_pipeline__FLAG_OWN_CAR,num_pipeline__FLAG_OWN_REALTY,num_pipeline__CNT_CHILDREN,num_pipeline__AMT_INCOME_TOTAL,num_pipeline__AMT_CREDIT,num_pipeline__AMT_ANNUITY,num_pipeline__AMT_GOODS_PRICE,num_pipeline__REGION_POPULATION_RELATIVE,num_pipeline__DAYS_BIRTH,...,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MAX,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MEAN,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_SUM,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_VAR,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MIN,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MAX,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MEAN,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_SUM,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_VAR,num_pipeline__CC_COUNT
0,-1.275659,-0.692704,-0.671013,-0.594760,0.262863,0.028207,0.659683,-0.195843,4.002670,0.412766,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
1,0.783909,-0.692704,-0.671013,-0.594760,-0.193769,-0.669515,-1.195607,-0.548653,-0.424047,-1.698009,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
2,-1.275659,1.443619,-0.671013,-0.594760,-0.053267,-0.825277,-0.381114,-0.849195,0.194676,1.169480,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
3,0.783909,-0.692704,-0.671013,2.142663,0.016984,-0.840354,-1.113465,-0.953732,-0.974830,0.303968,...,8.493383,4.099667e-01,1.577291,0.761883,0.0,0.0,0.0,0.0,0.0,3.210057e+00
4,0.783909,-0.692704,-0.671013,-0.594760,0.087235,-0.348477,-0.780278,-0.195843,0.482243,-1.445848,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-9.316309e-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215249,0.783909,-0.692704,-0.671013,-0.594760,-0.027274,-0.167317,-0.457648,-0.156642,-0.264347,-1.122470,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-1.669192e+00
215250,0.783909,-0.692704,-0.671013,-0.594760,-0.165669,-0.864515,-0.717599,-0.849195,-0.996586,0.442923,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
215251,0.783909,1.443619,-0.671013,-0.594760,-0.053267,-0.348477,0.641209,-0.195843,0.482243,-0.596806,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-1.555721e+00
215252,0.783909,-0.692704,-0.671013,-0.594760,-0.018142,-0.752402,-0.648983,-0.757726,-1.051168,1.476156,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16


In [77]:
X_test_ = X_test.replace((np.inf, -np.inf), np.nan).reset_index(drop=True)
X_test_scaled = preprocessor.transform(X_test_)
feature_names_X_test = preprocessor.get_feature_names_out ()
X_test = pd.DataFrame(X_test_scaled, columns=feature_names_X_test)
X_test

,num_pipeline__CODE_GENDER,num_pipeline__FLAG_OWN_CAR,num_pipeline__FLAG_OWN_REALTY,num_pipeline__CNT_CHILDREN,num_pipeline__AMT_INCOME_TOTAL,num_pipeline__AMT_CREDIT,num_pipeline__AMT_ANNUITY,num_pipeline__AMT_GOODS_PRICE,num_pipeline__REGION_POPULATION_RELATIVE,num_pipeline__DAYS_BIRTH,...,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MAX,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MEAN,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_SUM,num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_VAR,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MIN,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MAX,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MEAN,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_SUM,num_pipeline__CC_NAME_CONTRACT_STATUS_nan_VAR,num_pipeline__CC_COUNT
0,-1.275659,1.443619,1.490284,-0.594760,0.087235,-0.711083,-0.296992,-0.718525,0.655608,1.515361,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
1,-1.275659,-0.692704,-0.671013,0.773952,-0.137568,-1.312065,-1.547268,-1.306542,0.194676,0.606004,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
2,0.783909,-0.692704,-0.671013,-0.594760,-0.067317,-0.883540,-1.067611,-0.783860,0.859127,1.203813,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
3,0.783909,-0.692704,-0.671013,-0.594760,-0.053267,-1.327094,-1.702975,-1.319609,0.815308,-1.194383,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
4,0.783909,-0.692704,-0.671013,-0.594760,-0.074342,-0.526831,-0.572779,-0.391849,-0.156175,-1.682467,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92248,0.783909,1.443619,-0.671013,-0.594760,-0.053267,-0.140707,-0.323383,-0.130508,0.048717,-1.286943,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
92249,-1.275659,1.443619,-0.671013,0.773952,-0.088393,-0.348477,-0.370227,-0.195843,0.388500,0.266387,...,0.000000,-2.420554e-17,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,4.031296e-16
92250,-1.275659,1.443619,1.490284,-0.594760,-0.144593,-0.336587,-0.991077,-0.182776,-0.370534,-1.051253,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-8.181600e-01
92251,0.783909,-0.692704,-0.671013,-0.594760,0.016984,1.435064,1.528938,1.764214,0.482243,-0.554354,...,-0.402033,-2.033759e-01,-0.156366,-0.272536,0.0,0.0,0.0,0.0,0.0,-1.442250e+00


In [78]:
explainer = shap.LinearExplainer(model,  X_train)

In [80]:
X_test.iloc[0,:]

num_pipeline__CODE_GENDER                        -1.275659e+00
num_pipeline__FLAG_OWN_CAR                        1.443619e+00
num_pipeline__FLAG_OWN_REALTY                     1.490284e+00
num_pipeline__CNT_CHILDREN                       -5.947598e-01
num_pipeline__AMT_INCOME_TOTAL                    8.723501e-02
                                                      ...     
num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MAX     0.000000e+00
num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MEAN    0.000000e+00
num_pipeline__CC_NAME_CONTRACT_STATUS_nan_SUM     0.000000e+00
num_pipeline__CC_NAME_CONTRACT_STATUS_nan_VAR     0.000000e+00
num_pipeline__CC_COUNT                            4.031296e-16
Name: 0, Length: 795, dtype: float64

In [83]:
X_test.iloc[0,:].index

Index(['num_pipeline__CODE_GENDER', 'num_pipeline__FLAG_OWN_CAR',
       'num_pipeline__FLAG_OWN_REALTY', 'num_pipeline__CNT_CHILDREN',
       'num_pipeline__AMT_INCOME_TOTAL', 'num_pipeline__AMT_CREDIT',
       'num_pipeline__AMT_ANNUITY', 'num_pipeline__AMT_GOODS_PRICE',
       'num_pipeline__REGION_POPULATION_RELATIVE', 'num_pipeline__DAYS_BIRTH',
       ...
       'num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MAX',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_MEAN',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_SUM',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_Signed_VAR',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MIN',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MAX',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_nan_MEAN',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_nan_SUM',
       'num_pipeline__CC_NAME_CONTRACT_STATUS_nan_VAR',
       'num_pipeline__CC_COUNT'],
      dtype='object', length=795)

In [145]:
shap_values = explainer.shap_values(X_test)

In [147]:
shap.bar_plot(shap_values[0], max_display=10, show=False, feature_names=X_test.columns)
fig = plt.gcf()
fig.suptitle('Explainer ', fontsize=16)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [141]:
shap.waterfall_plot(shap_values[0,:], max_display=10, show=False)
fig = plt.gcf()
fig.suptitle('Explainer ', fontsize=16)

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'

In [143]:
shap.waterfall_plot(shap_values[0,:])

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'

In [127]:
shap.plots.waterfall(shap_values[0])

AttributeError: 'numpy.ndarray' object has no attribute 'base_values'

In [2]:
print("SHAP Version : {}".format(shap.__version__))

SHAP Version : 0.41.0


In [3]:
print("SHAP Version : {}".format(np.__version__))

SHAP Version : 1.23.1
